```{autolink-concat}
```

<!-- no-set-nb-cells -->

# Symbolic amplitude models

Amplitude analysis is a method that is used intensively in particle and hadron physics experiments. The method allows us to describes the intensity distributions obtained from the experiments with the use of amplitude models. Amplitude models allow us to extract parameters about intermediate states appearing in the scattering processes, which are governed by the electroweak force and the strong force.

The complicated nature of the strong force, described by Quantum Chromodynamics, makes it difficult to derive intensity models from first principles. Instead, we have to rely on approximations given specific assumptions for the scattering process that we study. Each amplitude model that we formulate, is almost always merely an approximation of the true scattering process. As a consequence, we always have to reassess our analysis results and try alternative models. In addition, amplitude models can be extremely complicated, with large, complex-valued parametrizations and dozens of input parameters. We therefore want to evaluate these models with as much information as possible. That means large input data samples and 'fits' using the full likelihood function, which provides us a multidimensional description of the data by using event-based, unbinned fit methods.

Given these challenges, we can identify **three major requirements that amplitude analysis software should satisfy**:

:::{card} {material-regular}`speed` Performance
:link: performance
:link-type: ref
We want to evaluate likelihood functions as fast as possible over large data samples, so that we can optimize our model parameters by testing several hypotheses in due time.
:::

:::{card} {material-regular}`draw` Flexibility
:link: flexibility
:link-type: ref
We want to quickly formulate a wide range of amplitude models, given the latest theoretical and experimental insights.
:::

:::{card} {material-regular}`school` Transparency
:link: transparency
:link-type: ref
It should be easy to inspect the implemented amplitude models, ideally by using mathematical formulas, so that the analysis can easily be reproduced or compared to results from other experiments, tools, or theoretical models.
:::

---

(performance)=
## {material-regular}`speed` Performance

(array-oriented)=
### Array-oriented programming

Even though Python is a popular programming language for data science, it is too slow for performing computations over large data samples. Computations in Python programs are therefore almost always outsourced through third-party Python libraries that are written in C++ or other compiled languages. This leads to an **array-oriented programming** style. Variables represent multidimensional arrays and the computational backend performs the element-wise operations behind-the-scenes. This has the additional benefit that the higher level Python code becomes more readable.

In the following example, we have two data samples $a$ and $b$, each containing a million data points, and we want to compute $c_i=a_i+b_i^2$ for each of these data point&nbsp;$i$. For simplicity, we set both $a$ and $b$ to be `[0, 1, 2, ..., 999_999]`.

In [ ]:
a_lst = list(range(1_000_000))
b_lst = list(range(1_000_000))

#### Pure Python loop

Naively, one could compute $c$ for each data point by creating a list and filling it with $c_i = a_i+b_i^2$.

In [ ]:
%%timeit
c_lst = []
for a_i, b_i in zip(a_lst, a_lst):
    c_lst.append(a_i + b_i**2)

`for` loops like these are a natural choice when coming from compiled languages like C++, but are considerably much slower when done with Python.

#### Equivalent computation with arrays

[NumPy](https://numpy.org) is one of the most popular array-oriented libraries for Python. The data points for $a$ and $b$ are now represented by array objects...

In [ ]:
import numpy as np

a = np.array(a_lst)
b = np.array(b_lst)

...and the _array-oriented_ computation of $c = a+b^2$ becomes much **faster** and **more readable**.

In [ ]:
%%timeit
c = a + b**2

### Accelerated computational libraries

The 2010s saw the release of a number of Python packages for highly optimized numerical computational backends that gained popularity within the data science and Machine Learning communities. Examples of these are [Numba](https://numba.pydata.org) (2012), [TensorFlow](https://tensorflow.org) (2015), [Pytorch](https://pytorch.org) (2016), and [JAX](https://jax.rtfd.io) (2018). Just like NumPy, the core of these packages is written in highly performant languages like C++, but apply several smart techniques to make the computations even faster. The main techniques that these backends apply are:

- **Just-In-Time Compilation** (JIT): Python code is compiled if and only if it is run. JIT not only offers performance in a dynamic workflow, but also allows the compiler to optimize the code at runtime based on the actual data input.
- **Hardware acceleration**: JIT compilation is performed through an intermediate, device-agnostic layer of code (particularly [XLA](https://openxla.org/xla)), which allows the user to run their code not only on regular CPUs, but also on different types of hardware accelerators, like GPUs and TPUs.
- **Parallelization**: array-oriented computations can automatically parallelized over multiple CPU cores (multithreading) or multiple CPU, GPU or TPU devices (multiprocessing).
- **Automatic Differentiation**: Many of these libraries can automatically compute derivatives, which is useful for gradient-based optimization algorithms. While this functionality was designed with linear Machine Learning models in mind, it can be used to compute exact gradients over mathematical models.

These techniques are usually directly available with minor changes to existing [array-oriented code](#array-oriented). In most cases, it is just a matter of decorating the array-oriented function with a JIT-compile decorator and, where needed, replacing the calls to vectorized functions (such as summing up a column in two-dimensional array) with their accelerated equivalents.

::::{tab-set}

:::{tab-item} Original
```python
import numpy as np

def my_func(a, b):
    return np.sum(a + b**2, axis=0)
```
:::

:::{tab-item} Numba
```python
import numba as nb
import numpy as np

@nb.jit(nopython=True)
def my_func(a, b):
    return np.sum(a + b**2, axis=0)
```
:::

:::{tab-item} JAX
```python
import jax
import jax.numpy as jnp

@jax.jit
def my_func(a, b):
    return jnp.sum(a + b**2, axis=0)
```
:::

:::{tab-item} TensorFlow
```python
import tensorflow as tf
import tensorflow.experimental.numpy as tnp

@tf.function(jit_compile=True)
def my_func(a, b):
    return tnp.sum(a + b**2, axis=0)
```
:::

:::{tab-item} Pytorch
```python
import torch

@torch.jit.script
def my_func(a, b):
    return torch.sum(a + b**2, dim=0)
```
:::

::::

As can be seen, the implementation of the array-oriented NumPy function remains largely unaffected by the switch to these accelerated computational libraries. The resulting JIT-compiled function objects are automatically compiled and parallelized for the selected device for fast numerical computations over large data samples.

:::{topic} {material-regular}`speed` Performance ✔️
Array-oriented programming allows for concise, recognizable implementations of mathematical models. Accelerated libraries like JAX and Numba can transform these implementations so that high-performance numerical computing can be achieved with trivial changes to the code.
:::

---

(flexibility)=
## {material-regular}`draw` Flexibility

Python offers us flexibility by running code interactively through a terminal or with Jupyter Notebooks. And as we saw, [array-oriented computational backends](#array-oriented) makes this code suitable for high-performance, parallelized computations over large data samples. The fact that array-oriented code looks so similar for different accelerated computational libraries now begs the question whether we can find a way to **directly convert the mathematical expressions that we as physicists are familiar with into these fast numerical functions**.

### Computer Algebra System

:::{todo}
Short explanation of SymPy, the Computer Algebra System for Python, with some simple examples.
:::

### Symbolic expressions as template

:::{todo}
- Expression tree visualizations (use previous symbolic examples)
- Code generation with SymPy expressions
- Example: the LaTeX rendering of the expressions is in itself achieved with code generation
- Example with generated [array-oriented](#array-oriented) code
:::

:::{topic} {material-regular}`draw` Flexibility ✔️
Summary box for this section.
:::

---

(transparency)=
## {material-regular}`school` Transparency

### Self-documenting workflow

:::{todo}
- Formulating amplitude models symbolically for numerical computations gives us the option to directly render them.
- Explanation of how this is ideal when working with Jupyter Notebooks (perhaps also a word on the popularity of notebooks, see also [Google Colab](https://colab.google) and a company like [Curvenote](https://curvenote.com).
- Explanation of the [Sphinx](https://www.sphinx-doc.org) ecosystem and plugins from the [Executable Book Project](https://executablebooks.org) make it possible to directly publish notebooks on the web or as [publication-ready PDF files](https://mystmd.org/overview/gallery).
:::

### Model preservation

:::{todo}
- Code generation can also be used to serialize symbolic amplitude models.
- Mathematics is the language we all speak: expressions that are rendered through published analysis code with the existing implementation, can always be implemented in other upcoming languages or with an alternative CAS.
:::

### Knowledge sharing

:::{todo}
- Find a better word.
- Explain why the amplitude analysis community needs to make the theory more understandable.
- Illustrate how the [self-documenting workflow](#self-documenting-workflow) makes it more inviting to contribute to community documentation as it narrows the gap between theory and code.
:::

:::{topic} {material-regular}`school` Transparency ✔️
Summary box for this section.
:::

---

## Summary

:::{todo}
For now, the points below are a collection of thoughts while writing the sections above.
:::

We believe that formulating amplitude models symbolically with a Computer Algebra System has several benefits for the amplitude analysis community:

- **Amplitude analyses become reproducible, extendable, and portable:**
  - Implemented amplitude models are transparently shared as mathematical formulas in a [self-documenting workflow](#self-documenting-workflow). This allows others to reimplement those models with their own framework of choice, or any time in the future when upcoming languages or libraries make the implementation of the analysis outdated.
  - The Python ecosystem in combination with Jupyter Notebooks and Sphinx makes it possible to directly rerun analysis in the browser or in some virtual environment locally. [Pinned dependencies](https://github.com/ComPWA/update-pip-constraints) ensure that the analysis produces the same results.

- **Lower entry level and knowledge sharing**:<br>
  It becomes much easier to share and maintain knowledge gained about amplitude models and amplitude analysis theory. Symbolic amplitude models directly show the implemented mathematics and their numerical functions can directly be used for interactive visualizations.